# CNN LSTM

**Reference:** https://github.com/vinayakumarr/Network-Intrusion-Detection

**Dataset:** http://kdd.ics.uci.edu/databases/kddcup99/kddcup99.html

In [1]:
import warnings
import numpy as np
np.random.seed(1337)  # for reproducibility

import pandas as pd

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Dense, Dropout, MaxPooling1D, LSTM

from sklearn.preprocessing import Normalizer

warnings.simplefilter('ignore')

In [2]:
from utils.preprocess import train_and_evaluate_model

def build_model(activation):
    model = Sequential()
    model.add(Conv1D(2, 5, padding="same", activation=activation, input_shape=X_train.shape[1:]))
    model.add(MaxPooling1D(1))
    model.add(LSTM(1))
    model.add(Dropout(0.1))
    model.add(Dense(4, activation="softmax"))
    model.compile(loss="sparse_categorical_crossentropy", optimizer="adam",metrics=['accuracy'])
    return model

In [3]:
DATA = '../data/'
MODELS = '../models/'
TEST = 0.3

In [4]:
traindata = pd.read_csv(DATA+'kddcup.data.csv')
traindata

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,Y
0,0,0,0,0,215,45076,0,0,0,0,...,0,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.0,0
1,0,0,0,0,162,4528,0,0,0,0,...,1,1.0,0.0,1.00,0.00,0.0,0.00,0.0,0.0,0
2,0,0,0,0,236,1228,0,0,0,0,...,2,1.0,0.0,0.50,0.00,0.0,0.00,0.0,0.0,0
3,0,0,0,0,233,2032,0,0,0,0,...,3,1.0,0.0,0.33,0.00,0.0,0.00,0.0,0.0,0
4,0,0,0,0,239,486,0,0,0,0,...,4,1.0,0.0,0.25,0.00,0.0,0.00,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4898426,0,0,0,0,212,2288,0,0,0,0,...,255,1.0,0.0,0.33,0.05,0.0,0.01,0.0,0.0,0
4898427,0,0,0,0,219,236,0,0,0,0,...,255,1.0,0.0,0.25,0.05,0.0,0.01,0.0,0.0,0
4898428,0,0,0,0,218,3610,0,0,0,0,...,255,1.0,0.0,0.20,0.05,0.0,0.01,0.0,0.0,0
4898429,0,0,0,0,219,1234,0,0,0,0,...,255,1.0,0.0,0.17,0.05,0.0,0.01,0.0,0.0,0


In [5]:
from utils import preprocess
traindata,testdata=preprocess.preprocess(traindata)

In [6]:
X = traindata.drop(columns=['Y'])
Y = traindata['Y']
C = testdata['Y']
T = testdata.drop(columns=['Y'])

In [7]:
scaler = Normalizer().fit(X)
X_train = scaler.transform(X)
X_test = scaler.transform(T)

In [8]:
y_train=Y
y_test= C

In [9]:
# reshape input to be [samples, time steps, features]
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

In [10]:
# define optimizer and objective, compile model
relu_model=build_model("relu")
tanh_model=build_model("tanh")

In [11]:
# train
history,report=train_and_evaluate_model(relu_model,X_train,y_train,X_test,y_test)

Epoch 1/4
2512/2512 [==============================] - 24s 8ms/step - loss: 1.0672 - accuracy: 0.5733 - val_loss: 1.0176 - val_accuracy: 0.5726
Epoch 2/4
2512/2512 [==============================] - 18s 7ms/step - loss: 1.0196 - accuracy: 0.5734 - val_loss: 1.0169 - val_accuracy: 0.5726
Epoch 3/4
2512/2512 [==============================] - 18s 7ms/step - loss: 0.9437 - accuracy: 0.6148 - val_loss: 0.6096 - val_accuracy: 0.7917
Epoch 4/4
2512/2512 [==============================] - 19s 8ms/step - loss: 0.5687 - accuracy: 0.7825 - val_loss: 0.3822 - val_accuracy: 0.8832
Evaluating Model: 
1436/1436 [==============================] - 5s 4ms/step - loss: 0.3821 - accuracy: 0.8833


In [12]:
print(report)

              precision    recall  f1-score   support

           0       1.00      0.46      0.63    291835
           1       0.00      0.00      0.00     13724
           2       0.72      1.00      0.84    321605
           3       0.95      1.00      0.97    842366

    accuracy                           0.88   1469530
   macro avg       0.67      0.61      0.61   1469530
weighted avg       0.90      0.88      0.87   1469530



In [13]:
tanh_hist, tanh_report = train_and_evaluate_model(tanh_model, X_train, y_train, X_test, y_test)

Epoch 1/4
2512/2512 [==============================] - 20s 7ms/step - loss: 0.7189 - accuracy: 0.7478 - val_loss: 0.3304 - val_accuracy: 0.9803
Epoch 2/4
2512/2512 [==============================] - 19s 7ms/step - loss: 0.3418 - accuracy: 0.9055 - val_loss: 0.2119 - val_accuracy: 0.9831
Epoch 3/4
2512/2512 [==============================] - 19s 7ms/step - loss: 0.2913 - accuracy: 0.9074 - val_loss: 0.1645 - val_accuracy: 0.9834
Epoch 4/4
2512/2512 [==============================] - 19s 7ms/step - loss: 0.2683 - accuracy: 0.9077 - val_loss: 0.1410 - val_accuracy: 0.9850
Evaluating Model: 
1436/1436 [==============================] - 5s 3ms/step - loss: 0.1419 - accuracy: 0.9849


In [14]:
print(tanh_report)

              precision    recall  f1-score   support

           0       0.99      0.98      0.98    291835
           1       0.00      0.00      0.00     13724
           2       0.95      1.00      0.97    321605
           3       1.00      1.00      1.00    842366

    accuracy                           0.98   1469530
   macro avg       0.73      0.74      0.74   1469530
weighted avg       0.98      0.98      0.98   1469530



In [15]:
relu_model.save("../models/relu_cnn_lstm.hdf5")
tanh_model.save("../models/tanh_cnn_lstm.hdf5")